# Treasure Hunt using Reinforcement Learning

In [1]:
# Q-learning helper implementation
import qlearning

### Build the map

In [2]:
DANGER = "d"
AGENT = "A"
COINS = "c"
EMPTY = "-"
TREE = "t"
END = "E"


map = [
    [AGENT, DANGER, DANGER, TREE, EMPTY],
    [COINS, EMPTY, DANGER, EMPTY, EMPTY],
    [COINS, EMPTY, TREE, TREE, EMPTY],
    [DANGER, COINS, COINS, TREE, EMPTY],
    [COINS, EMPTY, COINS, COINS, EMPTY],
]

In [3]:
for row in map:
    print(' '.join(row))

A d d t -
c - d - -
c - t t -
d c c t -
c - c c -


### All possible actions
- The agent can move up, down, left and right

In [4]:
UP = 0
DOWN = 1
LEFT = 2
RIGHT = 3

ACTIONS = [UP, DOWN, LEFT, RIGHT]

### Initial state
- Starting position of the agent

In [5]:
start_state = qlearning.State(map=map, agent_pos=[0, 0])

### Learning to drive using Q-Learning
- We will decay the alpha rate with each episode
- When the environment is unknown agent will take random actions else it will choose the best action

In [6]:
import numpy as np
import random

random.seed(42)

# Hyper parameters
N_EPISODES = 50

MAX_EPISODE_STEPS = 100

MIN_ALPHA = 0.02

alphas = np.linspace(1.0, MIN_ALPHA, N_EPISODES) # learning rate
gamma = 1.0
eps = 0.2

q_table = dict()
# Gives the Q value for a state-action pair or for all actions, given a state	
def q(state, action=None):
    
    if state not in q_table:
        q_table[state] = np.zeros(len(ACTIONS))
        
    if action is None:
        return q_table[state]
    
    return q_table[state][action]

# We will choose an action randomly
def choose_action(state):
    if random.uniform(0, 1) < eps:
        return random.choice(ACTIONS) 
    else:
        return np.argmax(q(state))

In [7]:
for e in range(N_EPISODES):
    
    state = start_state
    total_reward = 0
    alpha = alphas[e]
    
    for _ in range(MAX_EPISODE_STEPS):
        action = choose_action(state)
        next_state, reward, done = qlearning.move(state, action)
        total_reward += reward
        
        q(state)[action] = q(state, action) + \
                alpha * (reward + gamma *  np.max(q(next_state)) - q(state, action))
        state = next_state
        if done:
            break
    print(f"Episode {e + 1}: Total coins = {total_reward}")

YOU ARE IN DANGER!!
Episode 1: Total coins = -53
YOU ARE IN DANGER!!
Episode 2: Total coins = -50
YOU ARE IN DANGER!!
Episode 3: Total coins = -3
YOU ARE IN DANGER!!
Episode 4: Total coins = -3
YOU ARE IN DANGER!!
Episode 5: Total coins = -5
YOU ARE IN DANGER!!
Episode 6: Total coins = -10
YOU ARE IN DANGER!!
Episode 7: Total coins = 39
YOU ARE IN DANGER!!
Episode 8: Total coins = -3
YOU ARE IN DANGER!!
Episode 9: Total coins = 0
YOU ARE IN DANGER!!
Episode 10: Total coins = 0
YOU ARE IN DANGER!!
Episode 11: Total coins = 0
YOU ARE IN DANGER!!
Episode 12: Total coins = -4
YOU ARE IN DANGER!!
Episode 13: Total coins = 42
YOU ARE IN DANGER!!
Episode 14: Total coins = -2
YOU ARE IN DANGER!!
Episode 15: Total coins = 88
YOU ARE IN DANGER!!
Episode 16: Total coins = -50
YOU ARE IN DANGER!!
Episode 17: Total coins = -100
YOU ARE IN DANGER!!
Episode 18: Total coins = -9
YOU ARE IN DANGER!!
Episode 19: Total coins = -1
YOU ARE IN DANGER!!
Episode 20: Total coins = -1
YOU ARE IN DANGER!!
Episod

### Extract the policy  agent has learned
- For the map where start location is (0,0) down is the best place to move as there are coins, so down has highest rewards
- Then we will move down and see what the new state looks like
- Again down has maximum rewards (spoiler - it must have coins!)
- It seems the agent knows its way around to get most coins

In [8]:
r = q(start_state)
print(f"up={r[UP]}, down={r[DOWN]}, left={r[LEFT]}, right={r[RIGHT]}")

up=87.31564653876889, down=202.46457177309412, left=75.64152644119244, right=-68.0


In [9]:
new_state, reward, done = qlearning.move(start_state, DOWN)

In [10]:
r = q(new_state)
print(f"up={r[UP]}, down={r[DOWN]}, left={r[LEFT]}, right={r[RIGHT]}")

up=49.978129487867086, down=169.97560127739277, left=46.362111967077205, right=0.0


### Conclusion
- Initially the agent does not know map elements
- Yet it manages to learn that "DANGER" is bad and "COINS" are good
- More we train, the agent will make optimal decisions